In [19]:
import h5py as h5py
import torch
from torch.nn.parameter import Parameter
import torch.nn as nn
import torch.nn.functional as F

filename = 'MNIST_synthetic.h5'
f = h5py.File(filename, 'r')

train_dataset = f['train_dataset'][...]
train_labels = f['train_labels'][...]
test_dataset = f['test_dataset'][...]
f.close()


Keys: <KeysViewHDF5 ['test_dataset', 'train_dataset', 'train_labels']>


In [ ]:
# NOT DONE

# Each layer in NN has 2 primary components : transformation and collection of weights
# transformation represented as code
# collection of weights represented as data
# pytorch layers are defined by classes
# Every pytorch nn.Module has a forward method that needs to be implemented
# When we're building layers, we need to implement forward. Forward is the transformation
# pytorch neural network

# CNN requires linear and convolution layers
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        # in_channels 3 because our data is RGB
        # if we decide to pre process our data to grayscale then it would be 1

        # kernel_size being 3 means that we're doing 3x3 square convolution
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3) # image dimension is 3xNxN

        # second convolutional layer, i.e. the output of the first layer will be the input of this layer
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3)

        # now that we've done some convolution, we want to flatten the output
        # and use it as input for linear layers

        # after we run our image through the first 2 convolutional layers
        # we get another 3D object that's 16 * x * x

        x = 1
        self.fc1 = nn.Linear(in_features=16 * x * x , out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)

        # output layer
        self.fc3 = nn.Linear(in_features=84, out_features=11) # 11 because we have 11 labels

    # Takes in tensor t and transform it using layer, new tensor is then returned
    def forward(self, x):
        # Picking our activation function for our cnn :
        # We decided to use ReLu instead of a sigmoid function.
        # Training a neural network is computationally expensive, ReLu is less expensive to compute
        # because it does not need to compute exponential operations like Sigmoid.
        # From stackoverflow:
        # The biggest advantage of ReLu is indeed non-saturation of its gradient,
        # which greatly accelerates the convergence of stochastic gradient descent compared to the sigmoid / tanh functions
        # http://www.cs.toronto.edu/~fritz/absps/imagenet.pdf

        # max pooling over a (2,2) window (window of 4 elements)
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)

        # before we can pass it into the fully connected layer, i.e. fc1,f2,fc3
        # we need to flatten it down to a single tensor
        x = x.view(-1, self.num_flat_features(x))

        # We run it through our linear layers now
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

    def num_flat_features(self, x):
        """
        Helper function
        :param x:
        :return: the number of flat features that we have
        """
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


In [ ]:
network = Network()
print(network)

In [ ]:
# For building CNN layers, we have the following hyperparameters :
# Kernel size : Sets the size of the filter inside the layer. Kernel = Filter
# Out channels : Sets the number of filters. One filter produces one output channel.
# Out features : Sets the size of the output tensor
# Inside a convolutional layer, the input channels are paired with a convolutional filter
# to perform the convolutional operation. The filter convolves the input channel and
# the result of this operation is an output channel.